**Análise Exploratório de Dados - Movielens 1M**

---

Neste Notebook você irá analisar o dataset Movilens 1M por meio de uma análise exploratória de dados. 

# Index

1. Configuração Inicial;

2. Enquadrar o problema;

3. Análise Exploratória de Dados ML1M-Cao; // Qualidade dos dados, artigo performance similar em modelos distintos, qualidades dos dados tinha maior impacto. Problema de SmallData?
    
    3.1. Obter os dados;
    
    3.2. Variáveis núméricas;
    
    3.3. Variáveis categóricas;
    
    3.4. Cleanning data;
    
    3.5. Data visualization;
           
4. Conjuntos de treinamento / correlação / featuring engineering;

5. Preparar os dados para os algoritmos;

6. Selecionar e treinar modelos; // Esta e demais etapas sao realizadas pelo projeto know-rec.

7. Ajustar o modelo;

8. Apresentar sua solução;

9. Lançar, monitorar e manter seu sistema.

# 1. Configuração inicial

In [2]:
# Importações comuns
import numpy as np
import os
import pandas as pd # pandas is a data manipulation library
import random
from wordcloud import WordCloud, STOPWORDS #used to generate world cloud

In [3]:
#Para garantir estabilidade e ser mais fácil reproduzir experimento
np.random.seed(42)

In [4]:
# Para plotar figuras
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

[Confira aqui](https://matplotlib.org/api/_as_gen/matplotlib.pyplot.rc.html) a documentação do matplotlib.rc

In [5]:
# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "end_to_end_project"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

In [6]:

# Ignorar warnings desnecessários (ver SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

# 2. Enquadar o problema

- Qual o objetivo do problema?

- Como a empresa/cliente pretende usar o produto?

Tais perguntas são importantes pois definirá como você vai abordar o problema, que tipo de algoritmo irá usar e quais [medidas de desempenho](http://geam.paginas.ufsc.br/files/2020/02/Medida-desempenho-regressaob.pdf) são mais relevantes. 

// Obs. 1: Comparar interpretação geográfica de cada métrica (MAE, MSE e RMSE). Plotar as 3 retas no mesmo gráfico.

Você precisará avaliar se a solução requer uma solução muito complexa, que demandará mais trabalho, tempo e dinheiro, ou se uma solução mais simples será suficiente.

Recomendação de filmes.

1) Predizer o valor da avaliação de usuário para filme assistido.

2) Classificar se um filme será ou não assistido pelo usuário.

3) Recomendar um grupo de filmes com maior possibilidade de escolha do usuário e maior diversidade entre os filmes.

4) Modelo com maior satisfação do cliente/empresa?!

// Navalha de Okan: se você tem 2 soluções que satisfazem o problema, deve ser escolhida a mais simples. (como propor deep learning e sistemas baseados em conhecimento? Quando eles são realmente necessários?

 ## Dicas

- Não aborde, em um primeiro momento, um problema usando a solução mais complexa possível. Otimização prematura é arriscado e pode comprometer o projeto;

- Leve em consideração que os modelos mais complexos são mais difíceis de manter, requer estruturas mais sofisticadas (e mais caras) e geralmente requer um corpo técnico mais qualificado - fique atento também as regulamentações dos dados;

- Comece com protótipos rápidos e vá conversando com o cliente obtendo retorno sobre as necessidades do produto. Já pensou passar meses desenvolvendo um produto e no final não era o que o cliente queria? A agilidade em fazer protótipos em Python torna essa linguagem muito interessante!

- As nossas visões, opiniões vão mudando com o tempo, então é natural que o cliente (e você!) vá amadurecendo ao longo do processo. Então, repetindo, sempre se comunique para atender a necessidade do projeto!

- Antes de começar a trabalhar no projeto, verifique todas as hipóteses do sistema, infraestrutura disponível, linguagens de programação que serão utilizada, plataformas, etc. 

# 3. Análise Exploratória de Dados - ML1M-Cao

## 3.1. Obter os dados

In [7]:
# https://www.kaggle.com/cesarcf1977/movielens-data-analysis-beginner-s-first
# from http://www.gregreda.com/2013/10/26/using-pandas-on-the-movielens-dataset/
CAO_PATH = os.path.join("..", "..", "datasets", "ml1m-cao")
def load_ml1m_cao_data(save_path=CAO_PATH):
    # pass in column names for each CSV
    # rating files: train, valid and test
    r_cols = ['user_id', 'item_id', 'rating']
    csv_path = os.path.join(save_path, "ml1m", "train.dat")
    train = pd.read_csv(csv_path, sep='\t', engine="python", names=r_cols, encoding='utf8', header=None)

    csv_path = os.path.join(save_path, "ml1m", "valid.dat")
    valid = pd.read_csv(csv_path, sep='\t', engine="python", names=r_cols, encoding='utf8', header=None)

    csv_path = os.path.join(save_path, "ml1m", "test.dat")
    test = pd.read_csv(csv_path, sep='\t', engine="python", names=r_cols, encoding='utf8', header=None)

    frames = [train, valid, test]
    ratings = pd.concat(frames)
    
    # kg files: train, valid and test
    kg_cols = ['sub_id', 'obj_id', 'pred_id']
    csv_path = os.path.join(save_path, "ml1m", "kg", "train.dat")
    kg_train = pd.read_csv(csv_path, sep='\t', engine="python", names=kg_cols, encoding='utf8', header=None)

    csv_path = os.path.join(save_path, "ml1m", "kg", "valid.dat")
    kg_valid = pd.read_csv(csv_path, sep='\t', engine="python", names=kg_cols, encoding='utf8', header=None)

    csv_path = os.path.join(save_path, "ml1m", "kg", "test.dat")
    kg_test = pd.read_csv(csv_path, sep='\t', engine="python", names=kg_cols, encoding='utf8', header=None)

    frames = [kg_train, kg_valid, kg_test]
    kg = pd.concat(frames)
    
    # map files: i_map, i2kg_map and e_map
    csv_path = os.path.join(save_path, "ml1m", "i_map.dat")
    u_cols = ['mapped_id', 'orig_id']
    i_map = pd.read_csv(csv_path, sep='\t', engine="python", names=u_cols, encoding='utf8', header=None)

    csv_path = os.path.join(save_path, "ml1m", "i2kg_map.tsv")
    r_cols = ['orig_id', 'name', 'uri']
    i2kg_map = pd.read_csv(csv_path, sep='\t', engine="python", names=r_cols, encoding='utf8', header=None)
    
    csv_path = os.path.join(save_path, "ml1m", "kg", "e_map.dat")
    r_cols = ['e_id', 'uri']
    e_map = pd.read_csv(csv_path, sep='\t', engine="python", names=r_cols, encoding='utf8', header=None)
    
    csv_path = os.path.join(save_path, "ml1m", "kg", "r_map.dat")
    r_cols = ['r_id', 'uri']
    r_map = pd.read_csv(csv_path, sep='\t', engine="python", names=r_cols, encoding='utf8', header=None)

    # create one merged DataFrame
    return train, valid, test, ratings, kg_train, kg_valid, kg_test, kg, i_map, i2kg_map, e_map, r_map

In [12]:
train, valid, test, ratings, kg_train, kg_valid, kg_test, kg, i_map, i2kg_map, e_map, r_map = load_ml1m_cao_data()
e_map.head()

,e_id,uri
0,0,http://dbpedia.org/resource/Roger_Carel
1,1,http://dbpedia.org/resource/Soundtrack_album
2,2,http://dbpedia.org/resource/1982_in_film
3,3,http://dbpedia.org/resource/Category:Films_set...
4,4,http://dbpedia.org/resource/Plaza_Hotel


In [9]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 998539 entries, 0 to 193630
Data columns (total 3 columns):
user_id    998539 non-null int64
item_id    998539 non-null int64
rating     998539 non-null int64
dtypes: int64(3)
memory usage: 30.5 MB


In [64]:
kg.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 434189 entries, 0 to 86834
Data columns (total 3 columns):
sub_id     434189 non-null int64
obj_id     434189 non-null int64
pred_id    434189 non-null int64
dtypes: int64(3)
memory usage: 13.3 MB


In [65]:
i_map.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3260 entries, 0 to 3259
Data columns (total 2 columns):
mapped_id    3260 non-null int64
orig_id      3260 non-null int64
dtypes: int64(2)
memory usage: 51.0 KB


In [66]:
i2kg_map.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3301 entries, 0 to 3300
Data columns (total 3 columns):
orig_id    3301 non-null int64
name       3301 non-null object
uri        3301 non-null object
dtypes: int64(1), object(2)
memory usage: 77.4+ KB


In [67]:
e_map.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14708 entries, 0 to 14707
Data columns (total 2 columns):
e_id    14708 non-null int64
uri     14708 non-null object
dtypes: int64(1), object(1)
memory usage: 229.9+ KB


In [71]:
r_map

,r_id,uri
0,0,http://dbpedia.org/ontology/cinematography
1,1,http://dbpedia.org/property/productionCompanies
2,2,http://dbpedia.org/property/composer
3,3,http://purl.org/dc/terms/subject
4,4,http://dbpedia.org/ontology/openingFilm
5,5,http://www.w3.org/2000/01/rdf-schema#seeAlso
6,6,http://dbpedia.org/property/story
7,7,http://dbpedia.org/ontology/series
8,8,http://www.w3.org/1999/02/22-rdf-syntax-ns#type
9,9,http://dbpedia.org/ontology/basedOn


Arquivos de ratings: train.dat, valid.dat e test.dat

Arquivos de Knowledge Graph (KG): kg/train.dat, kg/valid.dat e kg/test.dat

Arquivos de mapping: i_map.dat, i2kg_map.dat, e_map.dat, r_map.dat e u_map.dat  

ratings train.dat, valid.dat e test.dat format: user_id '\t' item_id '\t' rating
- user_id range between 0 and 6039 
- item_id range between 0 and 3259
- rating are made on a 5-star scale (whole-star ratings only)
- Each user has at least 17 ratings *** ML1M each user has at least 20 ratings

KG kg/train.dat, kg/valid.dat e kg/test.dat format: head_entity_id '\t' tail_entity_id '\t' relation_id
- head_entity_id (sub) range between 0 and 14707 
- tail_entity_id (obj) range between 1 and 14707
- relation_id (pred) range between 0 and 19
- 14708 entities, 3237 no_sub, 1616 no_obj, 9855 both
- Each sub has at least 1 triple
- Each pred has at least 2 triples
- Each obj has at least 1 triple

mapping u_map.dat and i_map.dat format: mapped_id '\t' original_id
- perfect matching on i_map x ratings and u_map x ratings

mapping kg/e_map.dat and kg/r_map.dat format: mapped_id '\t' entity_uri
- perfect matching on kg -> e_map and kg -> r_map

mapping i2kg_map.tsv format: original_id '\t' entity_title '\t' entity_uri
- 297 itens do i_map não tem correspondente no i2kg_map (left_only) e 338 itens do i2kg não aparecem no i_map
- 34 uris de i2kg_map não aparecem no e_map e 11442 uris de e_map não aparecem no i2kg_map

## 3.2. Variáveis numéricas

In [18]:
ratings.describe()

,user_id,item_id,rating
count,998539.000000,998539.000000,998539.000000
mean,3023.703927,1521.194224,3.582575
std,1728.486949,895.247958,1.116516
min,0.000000,0.000000,1.000000
25%,1506.000000,831.000000,3.000000
50%,3070.000000,1445.000000,4.000000
75%,4476.000000,2267.000000,4.000000
max,6039.000000,3259.000000,5.000000


In [20]:
ratings.groupby(['user_id']).count().describe()

,item_id,rating
count,6040.000000,6040.000000
mean,165.321026,165.321026
std,192.173701,192.173701
min,17.000000,17.000000
25%,44.000000,44.000000
50%,96.000000,96.000000
75%,207.000000,207.000000
max,2233.000000,2233.000000


In [14]:
kg.describe()

,sub_id,obj_id,pred_id
count,434189.000000,434189.000000,434189.000000
mean,7412.131477,7164.380065,10.131381
std,4247.698096,4269.502971,3.204016
min,0.000000,1.000000,0.000000
25%,3731.000000,3337.000000,8.000000
50%,7376.000000,7125.000000,12.000000
75%,11193.000000,10736.000000,12.000000
max,14707.000000,14707.000000,19.000000


In [21]:
kg.groupby(['sub_id']).count().describe()

,obj_id,pred_id
count,11471.000000,11471.000000
mean,37.851016,37.851016
std,49.230030,49.230030
min,1.000000,1.000000
25%,6.000000,6.000000
50%,13.000000,13.000000
75%,59.000000,59.000000
max,626.000000,626.000000


In [76]:
kg.groupby(['obj_id']).count().describe()

,sub_id,pred_id
count,13092.000000,13092.000000
mean,33.164452,33.164452
std,140.794338,140.794338
min,1.000000,1.000000
25%,8.000000,8.000000
50%,14.000000,14.000000
75%,28.000000,28.000000
max,4816.000000,4816.000000


In [23]:
kg.groupby(['pred_id']).count().describe()

,sub_id,obj_id
count,20.000000,20.000000
mean,21709.450000,21709.450000
std,63118.338429,63118.338429
min,2.000000,2.000000
25%,114.000000,114.000000
50%,1660.000000,1660.000000
75%,3179.750000,3179.750000
max,274408.000000,274408.000000


In [82]:
kg.groupby(['pred_id']).count().merge(r_map, left_on='pred_id', right_on='r_id')

,sub_id,obj_id,r_id,uri
0,1515,1515,0,http://dbpedia.org/ontology/cinematography
1,4463,4463,1,http://dbpedia.org/property/productionCompanies
2,2057,2057,2,http://dbpedia.org/property/composer
3,42043,42043,3,http://purl.org/dc/terms/subject
4,23,23,4,http://dbpedia.org/ontology/openingFilm
5,23,23,5,http://www.w3.org/2000/01/rdf-schema#seeAlso
6,132,132,6,http://dbpedia.org/property/story
7,2,2,7,http://dbpedia.org/ontology/series
8,88916,88916,8,http://www.w3.org/1999/02/22-rdf-syntax-ns#type
9,60,60,9,http://dbpedia.org/ontology/basedOn


In [45]:
i_map.describe()

,mapped_id,orig_id
count,3260.000000,3260.000000
mean,1629.500000,1995.654908
std,941.225265,1151.011094
min,0.000000,1.000000
25%,814.750000,1012.750000
50%,1629.500000,2041.500000
75%,2444.250000,2973.250000
max,3259.000000,3952.000000


In [46]:
i2kg_map.describe()

,orig_id
count,3301.000000
mean,1988.250227
std,1144.840724
min,2.000000
25%,1010.000000
50%,2023.000000
75%,2970.000000
max,3951.000000


In [48]:
e_map.describe()

,e_id
count,14708.000000
mean,7353.500000
std,4245.978215
min,0.000000
25%,3676.750000
50%,7353.500000
75%,11030.250000
max,14707.000000


In [49]:
# no matching cases left (i_map)
i_map.merge(ratings, how = 'outer', left_on='mapped_id', right_on='item_id' ,indicator=True).loc[lambda x : x['_merge']=='left_only'].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 0 entries
Data columns (total 6 columns):
mapped_id    0 non-null int64
orig_id      0 non-null int64
user_id      0 non-null int64
item_id      0 non-null int64
rating       0 non-null int64
_merge       0 non-null category
dtypes: category(1), int64(5)
memory usage: 104.0 bytes


In [50]:
# no matching cases left (ratings)
ratings.merge(i_map, how = 'outer', left_on='item_id', right_on='mapped_id' ,indicator=True).loc[lambda x : x['_merge']=='left_only'].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 0 entries
Data columns (total 6 columns):
user_id      0 non-null int64
item_id      0 non-null int64
rating       0 non-null int64
mapped_id    0 non-null int64
orig_id      0 non-null int64
_merge       0 non-null category
dtypes: category(1), int64(5)
memory usage: 104.0 bytes


In [33]:
# no matching cases left (i2kg_map)
i_map.merge(i2kg_map, how = 'outer' ,indicator=True).loc[lambda x : x['_merge']=='left_only'].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 297 entries, 0 to 3259
Data columns (total 5 columns):
mapped_id    297 non-null float64
orig_id      297 non-null int64
name         0 non-null object
uri          0 non-null object
_merge       297 non-null category
dtypes: category(1), float64(1), int64(1), object(2)
memory usage: 12.0+ KB


In [51]:
# no matching cases left (i2kg_map)
i2kg_map.merge(i_map, how = 'outer' ,indicator=True).loc[lambda x : x['_merge']=='left_only'].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 338 entries, 20 to 3298
Data columns (total 5 columns):
orig_id      338 non-null int64
name         338 non-null object
uri          338 non-null object
mapped_id    0 non-null float64
_merge       338 non-null category
dtypes: category(1), float64(1), int64(1), object(2)
memory usage: 13.6+ KB


297 itens do i_map não tem correspondente no i2kg_map (left_only) e 338 itens do i2kg não aparecem no i_map. 

In [52]:
# no matching cases left (i2kg_map)
i2kg_map.merge(e_map, how = 'outer' ,indicator=True).loc[lambda x : x['_merge']=='left_only'].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34 entries, 66 to 3287
Data columns (total 5 columns):
orig_id    34 non-null float64
name       34 non-null object
uri        34 non-null object
e_id       0 non-null float64
_merge     34 non-null category
dtypes: category(1), float64(2), object(2)
memory usage: 1.5+ KB


In [53]:
# no matching cases left (e_map)
e_map.merge(i2kg_map, how = 'outer' ,indicator=True).loc[lambda x : x['_merge']=='left_only'].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11442 entries, 0 to 14708
Data columns (total 5 columns):
e_id       11442 non-null float64
uri        11442 non-null object
orig_id    0 non-null float64
name       0 non-null object
_merge     11442 non-null category
dtypes: category(1), float64(2), object(2)
memory usage: 458.2+ KB


34 uris de i2kg_map não aparecem no e_map e 11442 uris de e_map não aparecem no i2kg_map.

In [57]:
# no matching cases left (kg)
kg.merge(e_map, how = 'outer', left_on='sub_id', right_on='e_id' ,indicator=True).loc[lambda x : x['_merge']=='left_only'].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 0 entries
Data columns (total 6 columns):
sub_id     0 non-null float64
obj_id     0 non-null float64
pred_id    0 non-null float64
e_id       0 non-null int64
uri        0 non-null object
_merge     0 non-null category
dtypes: category(1), float64(3), int64(1), object(1)
memory usage: 104.0+ bytes


In [58]:
# no matching cases left (e_map)
kg.merge(e_map, how = 'outer', left_on='obj_id', right_on='e_id' ,indicator=True).loc[lambda x : x['_merge']=='left_only'].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 0 entries
Data columns (total 6 columns):
sub_id     0 non-null float64
obj_id     0 non-null float64
pred_id    0 non-null float64
e_id       0 non-null int64
uri        0 non-null object
_merge     0 non-null category
dtypes: category(1), float64(3), int64(1), object(1)
memory usage: 104.0+ bytes


In [55]:
# no matching cases left (e_map)
e_map.merge(kg, how = 'outer', left_on='e_id', right_on='sub_id' ,indicator=True).loc[lambda x : x['_merge']=='left_only'].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3237 entries, 108 to 437411
Data columns (total 6 columns):
e_id       3237 non-null int64
uri        3237 non-null object
sub_id     0 non-null float64
obj_id     0 non-null float64
pred_id    0 non-null float64
_merge     3237 non-null category
dtypes: category(1), float64(3), int64(1), object(1)
memory usage: 155.0+ KB


In [56]:
# no matching cases left (e_map)
e_map.merge(kg, how = 'outer', left_on='e_id', right_on='obj_id' ,indicator=True).loc[lambda x : x['_merge']=='left_only'].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1616 entries, 0 to 435337
Data columns (total 6 columns):
e_id       1616 non-null int64
uri        1616 non-null object
sub_id     0 non-null float64
obj_id     0 non-null float64
pred_id    0 non-null float64
_merge     1616 non-null category
dtypes: category(1), float64(3), int64(1), object(1)
memory usage: 77.4+ KB


In [69]:
# no matching cases left (kg)
kg.merge(r_map, how = 'outer', left_on='pred_id', right_on='r_id' ,indicator=True).loc[lambda x : x['_merge']=='left_only'].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 0 entries
Data columns (total 6 columns):
sub_id     0 non-null int64
obj_id     0 non-null int64
pred_id    0 non-null int64
r_id       0 non-null int64
uri        0 non-null object
_merge     0 non-null category
dtypes: category(1), int64(4), object(1)
memory usage: 104.0+ bytes


In [70]:
# no matching cases left (e_map)
r_map.merge(kg, how = 'outer', left_on='r_id', right_on='pred_id' ,indicator=True).loc[lambda x : x['_merge']=='left_only'].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 0 entries
Data columns (total 6 columns):
r_id       0 non-null int64
uri        0 non-null object
sub_id     0 non-null int64
obj_id     0 non-null int64
pred_id    0 non-null int64
_merge     0 non-null category
dtypes: category(1), int64(4), object(1)
memory usage: 104.0+ bytes


## 3.3. Variáveis categóricas

In [ ]:
csv = os.path.join("..", "..", "datasets", "ml1m-cao2sun", "ml1m", "rating-delete-missing-itemid.txt")
rdmi = pd.read_csv(csv, sep='\t', engine="python", names=['u', 'mapped_id', 'r', 't'], encoding='utf8')
rdmi['mapped_id'].value_counts()

In [ ]:
df = e_map.merge(i2kg_map, how = 'outer' ,indicator=True).loc[lambda x : x['_merge']=='left_only'] 

df.describe()

In [ ]:
df = i2kg_map.merge(e_map, how = 'outer' ,indicator=True).loc[lambda x : x['_merge']=='left_only'] 

df.describe()

## 3.4. Cleanning data

## 3.5. Data visualization

In [ ]:
i2kg_map.describe()

In [ ]:
e_map.describe()

# 4. Separando o conjunto de dados

Estratificação dos ratings, pois é importante ter um número suficiente de instâncias para cada estrato no conjunto de dados (treino e testes), do contrário pode ser que os nossos dados fiquem enviasados.

In [ ]:
ratings["rating"].hist()

Pronto! Agora vamos fazer uma <font color='red'>amostragem estratificada</font> com base nas categorias da renda.  

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(housing, housing["income_cat"]):
    strat_train_set = housing.loc[train_index]
    strat_test_set = housing.loc[test_index]

Acabamos de criar novos conjuntos de treino e de teste, que chamamos de <font color='red'>strat_train_set </font> e <font color='blue'>strat_test_set</font>.

 Estes conjuntos devem respeitar a estratificação que introduzimos baseada em "median_income" representado na nova variável categórica "income_cat".

 Vejamos se funcionou:

In [ ]:
strat_test_set["income_cat"].value_counts() / len(strat_test_set) #Proporção de cada categoria em strat_test_set

In [ ]:
housing["income_cat"].value_counts() / len(housing) #Proporção de cada categoria em housing

Podemos agora comparar com a <font color='blue'> amostragem aleatória </font>:

In [ ]:
#Função para calcular as proporções das categorias da característica "income_cat"
def income_cat_proportions(data): 
    return data["income_cat"].value_counts() / len(data)

Agora vamos gerar novamente conjunto de teste e treino, mas usando amostragem aleatória.

In [ ]:
train_set, test_set = train_test_split(housing, test_size=0.2, random_state=42)

Vamos criar o nosso novo dataframe e visualizar os resultados:

In [ ]:
compare_props = pd.DataFrame({
    "Geral": income_cat_proportions(housing),
    "Estratificado": income_cat_proportions(strat_test_set),
    "Aleatorio": income_cat_proportions(test_set),
}).sort_index()

compare_props["Aleatório %erro"] = 100 * compare_props["Aleatorio"] / compare_props["Geral"] - 100
compare_props["Estratificado %erro"] = 100 * compare_props["Estratificado"] / compare_props["Geral"] - 100

compare_props

Contentes com os resultados, não podemos esquecer de <font color='red'>remover</font> o atributo "income_cat" dos conjuntos strat_train_set e strat_test_set. Na verdade, ele era apenas um intermediário, afinal de contas as informações dessa caracaterísticas já estão presentes em "median_income".

In [ ]:
for set_ in (strat_train_set, strat_test_set):
    set_.drop("income_cat", axis=1, inplace=True)

# Visualização da estrutura de dados

Vamos agora visualizar os nossos dados. Precisamos ter certeza que não vamos visualizar dados do conjunto de teste, para evitar enviesamento de conclusões.

In [ ]:
housing = strat_train_set.copy() #Importante criar uma cópia! 

In [ ]:
housing.plot(kind="scatter", x="longitude", y="latitude")

Vamos melhorar a visualição usando o parâmetro <font color='red'>alpha</font>, observe:

In [ ]:
housing.plot(kind="scatter", x="longitude", y="latitude", alpha=0.1)

Interessante! Agora fica mais evidente a concentração dos agrupamentos!

De qualquer forma devemos voltar a nossa atenção ao objetivo: <font color = 'red'> preços do setor imobioliário. </font> 

No código a seguir o parâmetro "s" significa "size", tamanho em inglês. Escolhendo "s" como sendo a característica população, quanto maior o disco representa uma população maior.

O parâmetro "c" significa "color", ou cor. Esse é na verdade o que queremos saber!

O paramêtro colorbar = True indica que queremos visualizar a barra lateral informando as intensidades da cor, ou seja, do parêmetro "c".



In [ ]:
housing.plot(kind="scatter", x="longitude", y="latitude", alpha=0.4,
    s=housing["population"]/100, label="population", figsize=(10,7),
    c="median_house_value", cmap=plt.get_cmap("jet"), colorbar=True,
    sharex=False) #sharex=false é só pra corrigir um bug de display https://github.com/pandas-dev/pandas/issues/10611
plt.legend()

A visulização dos dados indicam que regiões litorâneas tendem a possuir um valor mais alto. Talevz a densidade populacional também possa ser algo relevante.

Vamos então investigar essas hipóteses através da correleção estatística:

In [ ]:
corr_matrix = housing.corr() #Matriz de correlações

In [ ]:
corr_matrix #vamos ver a estrutura

In [ ]:
corr_matrix["median_house_value"].sort_values(ascending=False) #Ordenar valores em sentido decrescente

É conveniente usar o scatter_matrix do pandas. Essa função plota cada característica em relação a outra. No nosso exemplo, teríamos 121 possibilidades.

Vamos aproveitar e ver alguns [conceitos básicos de estatística](http://geam.paginas.ufsc.br/files/2020/02/Estatistica_Basica.pdf).

Mas claro que não faremos isso e vamos então selecionar algumas que parecem ser mais significativas:

In [ ]:
from pandas.plotting import scatter_matrix

attributes = ["median_house_value", "median_income", "total_rooms",
              "housing_median_age"]
scatter_matrix(housing[attributes], figsize=(12, 8))

OBS: Na diagonal principal da plotagem anterior não temos atributo x atributo, mas sim o histograma da característica.

Vimos antes, que a característica que tinha maior correleção com o valor mediano de casas em um bairro era o salário mediano. Então vamos plotar para estudar a relação entre ambos:

In [ ]:
housing.plot(kind="scatter", x="median_income", y="median_house_value",
             alpha=0.1)
plt.axis([0, 16, 0, 550000])

Informações desta plotagem: 

1) Correlação é forte;

2) Há um valor limiar de 500.000 para os valores (medianos) das casas. Por quê?

3) Há também outras linhas horizontais. Por que elas são importantes?

Uma abordagem possível seria excluir os dados correspondentes a esses casos.

# Feature Engineering

Além das colunas que o conjunto de dados nos oferece, podemos tentar construir novas características <font color = "red">construídas de maneiras não linear</font> com as características existentes.

De maneira geral, essa etapa requer conhecimento específico da área na qual se esta trabalhando. Daí a importância da presença de um especialista no assunto para auxiliar no projeto. 

A seguir, vamos construir algumas novas features que são mais ou menos lógicas.

In [ ]:
#Nova feature: Número de cômodos por familia (média)
housing["rooms_per_household"] = housing["total_rooms"]/housing["households"]

#Nova feature: quartos/cômodos
housing["bedrooms_per_room"] = housing["total_bedrooms"]/housing["total_rooms"]

#Nova feature: população/agregado familiar
housing["population_per_household"]=housing["population"]/housing["households"]

Vejamos agora a matriz de correlação de housing:

In [ ]:
corr_matrix = housing.corr()
corr_matrix["median_house_value"].sort_values(ascending=False)

Aparentemente, casas com uma baixa proporção de quartos para cômodas tendem a ser mais caras. O número de cômodos por família é muito mais informartivo que o número total de quartos em um quarteirão.

Vejamos o gráfico:

In [ ]:
housing.plot(kind="scatter", x="rooms_per_household", y="median_house_value",
             alpha=0.2)
plt.axis([0, 5, 0, 520000])
plt.show()

Vamos ver novamente as medidas resumos considerando as novas features!

In [ ]:
housing.describe()

# Preparar os dados para os algoritmos de Machine Learning

Precisamos incialmente retirar os rótulos do conjunto <fon color='blue'> strat_train_set </font> (mais a frente ficará claro).

Para isso, vamos usar o método drop:

In [ ]:
housing = strat_train_set.drop("median_house_value", axis=1) # O método drop cria cópia sem a coluna em questao
housing_labels = strat_train_set["median_house_value"].copy() #salvando uma cópia

A partir de agora vamos partir para etapa de <font color='blue'>limpeza de dados!</font>

Vamos começar verificandi se temos dados falantes:

In [ ]:
#housing.isnull().any(axis=1) verifica quais linhas possuem alguma célula null
sample_incomplete_rows = housing[housing.isnull().any(axis=1)].head()
sample_incomplete_rows

Possuímos basicamente três abordagens possíveis para lidar com os dados faltantes:

1. Excluir os quarteirões com dados faltantes;

2. Excluir toda coluna de total_bedrooms, já que é o único atributo que apresenta dados faltantes;

3. Definir algum valor para substituir total_bedrooms.

In [ ]:
sample_incomplete_rows.dropna(subset=["total_bedrooms"])    # opção 1

In [ ]:
sample_incomplete_rows.drop("total_bedrooms", axis=1)       # opção 2

Opção 3: preenchendo com algum valor - nesse caso, usaremos a mediana.

In [ ]:
median = housing["total_bedrooms"].median()
sample_incomplete_rows["total_bedrooms"].fillna(median, inplace=True) # opção 3
sample_incomplete_rows

Se escolhermos a opção 3, devemos calular a mediana (ou qualquer outra medida que seja justificável) no <font color="red">conjunto de treinamento</font> e usá-lo para preencher os valores faltantes neste, mas precisamos <font color="blue">salvar</font> esse valor cálculado.

Você precisar desse valor para mais tarde aplicar no conjunto de teste, que deverá ter seus dados faltantes corrigidos seguindo o mesmo parâmetro do conjunto de treino.

**AVISO**: No Scikit-Learn 0.20, a classe `sklearn.preprocessing.Imputer` 
foi substituida pela classe `sklearn.impute.SimpleImputer`. Então, é conviniente verificar qual versão o computador em questão está usando:

In [ ]:
try:
    from sklearn.impute import SimpleImputer # Scikit-Learn 0.20+
    print("Scikit-Learn 0.20+")
except ImportError:
    from sklearn.preprocessing import Imputer as SimpleImputer
    print("Scikit-Learn antes do 0.20")

imputer = SimpleImputer(strategy="median")

Vamos novamente revisar o nosso dataset...

In [ ]:
housing

Ainda temos a última coluna que não é numérica! 

A princípios, grande parte dos algoritmos de machine learning no computador preferem os dados representados numericamente!

In [ ]:
housing_num = housing.drop('ocean_proximity', axis=1)
# Derrubando a coluna "ocean_proximity"
# alternativa: housing_num = housing.select_dtypes(include=[np.number])

Agora vamos ajudar o nosso objeto imputer com o nossos dados:

In [ ]:
imputer.fit(housing_num) 

Aqui, o imputer simplesmente calculou a mediana no conjunto de dados.

Vejamos algumas informações sobre o nosso objeto imputer:

In [ ]:
imputer.statistics_

Vamos verificar que isto é, na verdade, a mesma coisa que calcular manualmente a mediana de cada atributo:

In [ ]:
housing_num.median().values

Mas não seria apenas o atributo "total_bedrooms" que estava com valores faltantes? 

Vamos precisar de todas as informações do imputer? Isto é, vamos precisar da mediana de todas as variáveis?

<font color='red'> Não podemos, a princípio, afirmar que o mesmo padrão vai ser repetir na generalização do modelo! </font>

Certo, mas e se dermos uma espiadinha no conjunto de testes?

Não devemos fazer isso por vários motivos. 

1. Corremos o risco de colocar vieses no nosso modelo (assumir que apenas "total_bedrooms" terá colunas com dados faltantes em todos os cenários possíveis é um deles;

2. Devemos ter sempre em mente que o conjunto de teste é no fundo uma simulação para testarmos o poder de generalização do algoritmo - devemos fazer todas as nossas análises e otimizações somente no conjunto de treinamento e então aplicar o modelo final uma única vez no conjunto de teste!



Vamos agora finalmente <font color = 'blue'> transformar </font> o nosso conjunto de dados, aplicando, efetivamente, o valor calculado da mediana nos dados faltantes:

In [ ]:
X = imputer.transform(housing_num) #numpy array

Vamos visualizar o conjunto X

In [ ]:
X

Se você se sentir mais confortável, pode transformar o conjunto X em um dataframe:

In [ ]:
housing_tr = pd.DataFrame(X, columns=housing_num.columns, #importante informar nome das colunas
                          index=housing.index) #DataFrame Pandas

Vejamos como é este dataframe:

In [ ]:
housing_tr.head()

Agora devemos tratar a variável categórica`ocean_proximity'!

Lembre que esta é uma variável muito importante no nosso problema: ela demonstrava uma boa correlação com o preço mediano das casas.

Vamos novamente visualizar os dados para relembrar:

In [ ]:
housing_cat = housing[['ocean_proximity']]
housing_cat.head(10)

Agora vamos usar um processo chamado de codificação. Vamos transformar as nossas variáveis categóricas em números!

**OBS**: O código a seguir é apenas devido a atualização da classe OriginalEnconder()

In [ ]:
try:
    from sklearn.preprocessing import OrdinalEncoder
    print("Scikit-Learn >= 2.0")
except ImportError:
    from future_encoders import OrdinalEncoder # Scikit-Learn < 0.20
    print("O teu Scikit-Learn tá antiguinho mô quirido")

Na função a seguir, precisamos instanciar um objeto ordinal_encoder. 

Depois, usamos fit_transform para executa duas operações:

1. Método fit irá ajustar os parâmetros (mapeamento, por exemplo, quais são as variáveis categóricas); 

2. Método transform irá transformar os dados;

3. fit_transform(dados) irá ajustar parâmetros e transformar os dados.


In [ ]:
ordinal_encoder = OrdinalEncoder()
housing_cat_encoded = ordinal_encoder.fit_transform(housing_cat)

Uma alternativa mais prolixa teria sido escrever:

original_encoder.fit(housing_cat)

housing_cat_encoded = original_enconder.fit(housing_cat)


Vejamos que tipo de objeto é housing_cat_encoded:

In [ ]:
type(housing_cat_encoded)

Vamos ver agora os 10 primeiros valores desse numpy array:

In [ ]:
housing_cat_encoded[:10]

Vamos relembrar também as categorias do nosso problema:

In [ ]:
ordinal_encoder.categories_

Veja! 

O objeto ordinal_encoder foi construiído assim:

ordinal_encoder = OrdinalEncoder() 

e depois fizemos o seguinte:

housing_cat_encoded = ordinal_encoder.fit_transform(housing_cat)

<font color = "red">Aqui não apenas definimos quem é "housing_cat_encoded" como também inserimos informações no objeto ordinal_encoder! </font>

Apesar dos nossos esforços, temos um grave problema na nossa codificação, veja novamente: 

In [ ]:
housing_cat_encoded[:10]

In [ ]:
housing_cat[:10]

Cada variável categórica foi transformada em número!

Mas será que a princípio, podemos comparar uma variável categórica com outra?

Quem é maior: NEAR OCEAN ou NEAR BAY? 

Bem, é difícil responder. Mas é isso que a nossa codificação implicítacamente está fazendo ao colocar os valores 0,1,2,3 ou 4 para cada variável categórica. 

<font color="red"> Para lidar com essa situação precisamos então de outra abordagem!</font>

In [ ]:
try:
    from sklearn.preprocessing import OrdinalEncoder # gera um ImportError se Scikit-Learn < 0.20
    from sklearn.preprocessing import OneHotEncoder
except ImportError:
    from future_encoders import OneHotEncoder # Scikit-Learn < 0.20

cat_encoder = OneHotEncoder()
housing_cat_1hot = cat_encoder.fit_transform(housing_cat)
housing_cat_1hot

By default, the `OneHotEncoder` class returns a sparse array, but we can convert it to a dense array if needed by calling the `toarray()` method:

Epa! Agora temos uma matriz SciPy ao invés de um Numpy array! 

<font color = "red">Por que será?</font>

In [ ]:
housing_cat_1hot.toarray()

Temos agora uma matriz esparsa! (mais econômica computacionalmente)

Alternativamente, podemos colocar `sparse=False` ao criar o objeto `OneHotEncoder`:

In [ ]:
cat_encoder = OneHotEncoder(sparse=False)
housing_cat_1hot = cat_encoder.fit_transform(housing_cat)
housing_cat_1hot

In [ ]:
cat_encoder.categories_

Let's create a custom transformer to add extra attributes:

Vamos criar um transformador customizado para adicionar atributos extras 

**OBS**: aqui vamos simplesmente criar um código para o processo manual feito na etapa de Feature Engineering. Vai nos ajudar a criar um pipeline mais a frente.

In [ ]:
housing.columns

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

# Buscando os indices corretos das colunas: 
# Mais seeguro que ficar digitando 3, 4, 5, 6..
rooms_ix, bedrooms_ix, population_ix, household_ix = [
    list(housing.columns).index(col)
    for col in ("total_rooms", "total_bedrooms", "population", "households")]

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room = True): # no *args or **kwargs
        self.add_bedrooms_per_room = add_bedrooms_per_room
    def fit(self, X, y=None):
        return self  # Nada a fazer!
    def transform(self, X, y=None):
        rooms_per_household = X[:, rooms_ix] / X[:, household_ix]
        population_per_household = X[:, population_ix] / X[:, household_ix]
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            return np.c_[X, rooms_per_household, population_per_household,
                         bedrooms_per_room]
        else:
            return np.c_[X, rooms_per_household, population_per_household]

attr_adder = CombinedAttributesAdder(add_bedrooms_per_room=False)
housing_extra_attribs = attr_adder.transform(housing.values)

Alternativamente, você pode usar a função da classe `FunctionTransformer` que permite você criar rapidamente um transformador baseado em uma função de transformação! 

In [ ]:
from sklearn.preprocessing import FunctionTransformer

def add_extra_features(X, add_bedrooms_per_room=True):
    rooms_per_household = X[:, rooms_ix] / X[:, household_ix]
    population_per_household = X[:, population_ix] / X[:, household_ix]
    if add_bedrooms_per_room:
        bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
        return np.c_[X, rooms_per_household, population_per_household,
                     bedrooms_per_room]
    else:
        return np.c_[X, rooms_per_household, population_per_household]

attr_adder = FunctionTransformer(add_extra_features, validate=False,
                                 kw_args={"add_bedrooms_per_room": False})

housing_extra_attribs = attr_adder.fit_transform(housing.values)

#Vale a pena colocar validate=False já queos dados não possuem valores não-float
#validate=false é valor padrão a partir do Scikit-Learn 0.22.

In [ ]:
housing_extra_attribs = pd.DataFrame(
    housing_extra_attribs,
    columns=list(housing.columns)+["rooms_per_household", "population_per_household"],
    index=housing.index)
housing_extra_attribs.head()

Agora vamos construir um "pipeline" (tradução literal: gasoduto) para pré-processar os atributos numéricos - obser que poderíamos usar <font color = 'blue'> CombinedAttributesAdder()</font>
ao invés do <font color = 'blue'> FunctionTransformer(...) </font>, se quiséssemos:

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler 
#StandardScaler serve para fazer a reescalar das variáveis

num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")),
        ('attribs_adder', FunctionTransformer(add_extra_features, validate=False)),
        ('std_scaler', StandardScaler()),
    ])

housing_num_tr = num_pipeline.fit_transform(housing_num)

In [ ]:
housing_num_tr

In [ ]:
try:
    from sklearn.compose import ColumnTransformer
except ImportError:
    from future_encoders import ColumnTransformer # Scikit-Learn < 0.20

In [ ]:
num_attribs = list(housing_num)
cat_attribs = ["ocean_proximity"]

full_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_attribs),
        ("cat", OneHotEncoder(), cat_attribs),
    ])

housing_prepared = full_pipeline.fit_transform(housing)

In [ ]:
housing_prepared

In [ ]:
housing_prepared.shape

Agora finalmente temos os nossos dados pré-processados! 

# Selecionar e treinar um modelo

Vamos começar com um modelo siples: Regressão Linear!

In [ ]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(housing_prepared, housing_labels) 
#Ei Regressão linear, encontre os parâmetros que melhor aproxima os dados

Vamos agora testar o nosso pipeline de pré-processamento em algumas instâncias de treino.

In [ ]:
some_data = housing.iloc[:5]
some_labels = housing_labels.iloc[:5]
some_data_prepared = full_pipeline.transform(some_data) 

print("Predictions:", lin_reg.predict(some_data_prepared))

Vamos comparar agora com os valores reais:


In [ ]:
print("Labels:", list(some_labels))

In [ ]:
some_data_prepared

Agora vamos usar as métricas que aprendemos anteriormente!

In [ ]:
from sklearn.metrics import mean_squared_error as MSE

housing_predictions = lin_reg.predict(housing_prepared)
lin_mse = MSE(housing_labels, housing_predictions)
lin_rmse = np.sqrt(lin_mse) #Não é necessariamente obrigatório
lin_rmse

In [ ]:
from sklearn.metrics import mean_absolute_error as MAE

lin_mae = MAE(housing_labels, housing_predictions)
lin_mae

Essse modelo ainda não parece ser adequado!

In [ ]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor(random_state=42)
tree_reg.fit(housing_prepared, housing_labels)

In [ ]:
housing_predictions = tree_reg.predict(housing_prepared)
tree_mse = MSE(housing_labels, housing_predictions)
tree_rmse = np.sqrt(tree_mse)
tree_rmse

O quê? Erro zero?

#Vamos continuar na próxima aula a calibrar esse modelo!